In [ ]:
import os
from skimage import io
import matplotlib.pyplot as plt

from skimage.measure import regionprops
from skimage.morphology import label
from skimage.segmentation import expand_labels
from skimage.measure import regionprops
import numpy as np
from scipy import ndimage

In [ ]:
labels_path = r'D:\Institute of Genetics and Cancer\Neurosphere assay\Assess_Segmentation_Data\Ground_Truth'
images_path = r'D:\Institute of Genetics and Cancer\Neurosphere assay\Assess_Segmentation_Data\Images'

# Import test image
image_name = 'C - 02.tif'

img_filename = os.path.join(images_path, image_name)
lbl_filename = os.path.join(labels_path, image_name)
image = io.imread(img_filename)
labels = io.imread(lbl_filename)

# Get labels and plot
labels = label(labels)

fig, ax = plt.subplots(1, 3, figsize=(16, 8))
ax[0].imshow(image, cmap='gray')
ax[1].imshow(labels, cmap='nipy_spectral')
ax[2].imshow(image, cmap='gray')
ax[2].imshow(labels, cmap='nipy_spectral', alpha=.5)
fig.tight_layout

In [ ]:
import math

# Define additiinal properties
def circularity(p):
    return 4 * math.pi * p['area'] / p['perimeter']**2
def aspect_ratio(p):
    return p['axis_major_length'] / p['axis_minor_length']

# Get properties as numpy arrays
props = regionprops(labels)
area_array = np.asarray([p['area'] for p in props])
solidity_array = np.asarray([p['solidity'] for p in props])
circularity_array = np.asarray([circularity(p) for p in props])
aspect_ratio_array = np.asarray([aspect_ratio(p) for p in props])

In [ ]:
class_list = []
#bin_intervals=[0,250,625,900,10000]

for i in range(len(area_array)):
    label_id = props[i].label # gets the label id of the current region
    
    if(area_array[i]<250):
        class_mask = np.where(labels == label_id, 1, 0)
    elif(area_array[i]<750):
        class_mask = np.where(labels == label_id, 2, 0)
        class_mask.shape
    elif(area_array[i]<1250):
        class_mask = np.where(labels == label_id, 3, 0)
    else:
        class_mask = np.where(labels == label_id, 4, 0)
    
    class_list.append(class_mask)

# reconstructs the label image, now containing the eroded labels
class_stack = np.stack(class_list) # creates stack from list of images (numpy arrays)
image_classes = np.max(class_stack, axis = 0) # calculates the maximum projection to get back a 2D, labelled image

In [ ]:
from matplotlib import colors
cmap = colors.ListedColormap([u'k', u'tab:blue', u'tab:orange', u'tab:green', u'tab:red'])
#plt.imshow(image_classes, cmap=cmap, figsize=(8, 8))
#plt.colorbar(ticks=[0, 1, 2, 3, 4])
#plt.show()


fig, ax = plt.subplots(1, 3, figsize=(16, 8))
ax[0].imshow(image, cmap='gray')
ax[1].imshow(image_classes, cmap=cmap)
ax[2].imshow(image, cmap='gray')
ax[2].imshow(image_classes, cmap=cmap, alpha=.5)
#plt.colorbar(ticks=[0, 1, 2, 3, 4])

import matplotlib as mpl
cmap_colorbar = colors.ListedColormap([u'tab:blue', u'tab:orange', u'tab:green', u'tab:red'])
norm = colors.Normalize(vmin=1, vmax=4)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap_colorbar), ax=ax[:], ticks=[1, 2, 3, 4])

fig.tight_layout

In [ ]:
fig, ax = plt.subplots()
ax.imshow(image, cmap='gray')
ax.imshow(image_classes, cmap=cmap, alpha=.5)